# LASSO  with Coordinate Descent

In this notebook, we will implement LASSO regression via coordinate descent. We will:
* Write a function to normalize features
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write a coordinate descent function to compute the regression weights given an initial weight vector, step size and tolerance.

In [70]:
import turicreate
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [71]:
sales = turicreate.SFrame(r'home_data.sframe')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float and then to int, before using it below
sales['floors'] = sales['floors'].astype(float).astype(int)

# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in Turi Create functions), in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 
 
Now we will write a function that will accept an SFrame, a list of feature names and a target feature ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an intercept)
* A numpy array containing the values of the output

In [72]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe=data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray=data_sframe[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

We now define an additional function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [73]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions=np.dot(feature_matrix, weights)

    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, the weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features**: we divide each feature by its 2-norm so that the transformed feature has norm 1.

In [74]:
def normalize_features(feature_matrix):
    norms=np.linalg.norm(feature_matrix, axis=0)
    normalized_features=feature_matrix/norms
    
    return (normalized_features, norms)

# Implementing Coordinate Descent with Normalized Features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function (by convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function 
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. For each iteration:
1. Loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is below the tolerance, stop. Otherwise, go back to step 1. 

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

As an intermediary step, we will define a coordinate descent function that minimizes the cost function over a single feature i.

In [75]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    
    ro_i = (feature_matrix[:,i] * (output - prediction + (weights[i] * feature_matrix[:,i]))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [76]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False

    while not converged:

    # Evaluate over all features
        for idx in range(D):
#             print 'Feature: ' + str(idx)
            # new weight for feature
            new_weight = lasso_coordinate_descent_step(idx, feature_matrix,
                                                       output, weights,
                                                       l1_penalty)
            # compute change in weight for feature
            change[idx] = np.abs(new_weight - weights[idx])
#             print '  -> old weight: ' + str(weights[idx]) + ', new weight: ' + str(new_weight)
#             print '  -> abs change (new - old): ' + str(change[idx])
#             print '  >> old weights: ', weights

            # assign new weight
            weights[idx] = new_weight
#             print '  >> new weights: ', weights
        # maximum change in weight, after all changes have been computed
        max_change = max(change)
#        print '  ** max change: ' + str(max_change)
#         print '--------------------------------------------------'
        if max_change < tolerance:
            converged = True
    return weights

We now define our LASSO cyclical coordinate descent function:

In [77]:

def lasso_cyclical_coordinate_descent2(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights=np.array(initial_weights)
    dif=[tolerance+1]*len(initial_weights)
    while True:
        for i in range(len(initial_weights)):
            old=weights[i]
            weights[i]=lasso_coordinate_descent_step(i,feature_matrix,output,weights,l1_penalty)
            dif[i]=abs(weights[i]-old)
        if max(dif)<tolerance:
            break
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [78]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 10.0
tolerance = 1.0
max_iterations = 10000

Now create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [79]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [80]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights

array([ 1.16830179e+07,  1.04939550e+08, -2.93365075e+07])

# Evaluating LASSO Fit with More Features

Let us split the sales dataset into training and test sets.

In [82]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [83]:
all_features = ['bedrooms',  #zero
                'bathrooms', #nonzero
                'sqft_living', #nonzero
                'sqft_lot', #zero
                'floors', #zero 
                'waterfront', #nonzero
                'view', #nonzero
                'condition', #nonzero
                'grade', #zero
                'sqft_above', #nonzero
                'sqft_basement', #nonzero
                'yr_built', #nonzero
                'yr_renovated'] #zero

First, create a normalized feature matrix from the training data with these features.  (We store the norms for the normalization, since we'll use them later)

In [84]:
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_feature_matrix, norms) = normalize_features(feature_matrix) # normalize features

First, we learn the weights with `l1_penalty=1e7` on the training data.

In [85]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1.0
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights1e7

array([24429600.60933314,        0.        ,        0.        ,
       48389174.35227978,        0.        ,        0.        ,
        3317511.16271981,  7329961.9848964 ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

Next, we learn the weights with `l1_penalty=1e4`, on the training data. 

In [86]:
initial_weights = np.zeros(14)
l1_penalty = 1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights1e4

array([ 77779073.91265233, -22884012.25023365,  15348487.08089993,
        92166869.69883084,  -2139328.0824278 ,  -8818455.54409498,
         6494209.73310655,   7065162.05053197,   4119079.2100676 ,
        18436483.52618777, -14566678.54514345,  -5528348.75179428,
       -83591746.20730534,   2784276.46012858])

## Rescaling the Learned Weights

Recall that we normalized our feature matrix before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with the *original* features:
 1. Store the norms of the original features to a vector called `norms`:
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.

Now, we can apply `weights_normalized` to the test data, without normalizing it!

We create a normalized version of each of the weights learned above.

In [87]:
weights1e4_normalized=weights1e4/norms
weights1e7_normalized=weights1e7/norms

## Evaluate Each of the Learned Models on the Test Data

Let's now evaluate the three models on the test data by computing the RSS of the normalized weights on the (unnormalized) `test_feature_matrix`:

In [88]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [89]:
prediction =  predict_output(test_feature_matrix, weights1e7_normalized)
RSS_1e7 = np.dot(test_output-prediction, test_output-prediction)
print 'RSS for model with weights1e7 = ', RSS_1e7

RSS for model with weights1e7 =  275962079909185.25


In [90]:
prediction =  predict_output(test_feature_matrix, weights1e4_normalized)
RSS_1e4 = np.dot(test_output-prediction, test_output-prediction)
print 'RSS for model with weights1e4 = ', RSS_1e4

RSS for model with weights1e4 =  227781004760225.3


As we can see, the model with the smaller L1 penalty significantly performs its competitor.